In [136]:
# Player Similiarity Program
import pandas as pd
import numpy as np
import polars as pl
from api_scraper import MLB_Scrape

# Set display options to print all columns without truncation
pd.set_option("display.max_columns", None)  # Ensure all columns are displayed
pd.set_option("display.max_rows", None)  # Ensure all rows are displayed
pd.set_option("display.width", None)  # Remove column width limit

In [137]:
ab_flag_dict = {'Walk':0, 'Field Error': 1, 'Caught Stealing Home': 0, 'Strikeout Double Play': 1,
                'Sac Fly Double Play': 0,   'Hit By Pitch': 0, 'Runner Out': 0, 'Triple Play': 1, 
                'Field Out': 1, 'Flyout': 1, 'Bunt Lineout': 1, 'Catcher Interference': 0, 'Fielders Choice': 1,
                'Sac Bunt': 0, 'Pop Out': 1, 'Caught Stealing 2B': 0, 'Fielders Choice Out': 1, 'Stolen Base 2B': 0,
                'Sac Fly': 0, 'Groundout': 1, 'Home Run': 1, 'Caught Stealing 3B': 0, 'Intent Walk': 0, 'Double Play': 1,
                'Grounded Into DP': 1, 'Forceout': 1, 'Lineout': 1, 'Single': 1, 'Triple': 1, 'Wild Pitch': 0, 'Double': 1,
                'Strikeout': 1, 'Bunt Groundout': 0, 'Bunt Pop Out': 0}

ab_events = [event for event, flag in ab_flag_dict.items() if flag == 1]

pa_flag_dict = {'Walk':1, 'Field Error': 1, 'Caught Stealing Home': 0, 'Strikeout Double Play': 1,
                'Sac Fly Double Play': 1,   'Hit By Pitch': 1, 'Runner Out': 0, 'Triple Play': 1, 
                'Field Out': 1, 'Flyout': 1, 'Bunt Lineout': 1, 'Catcher Interference': 1, 'Fielders Choice': 1,
                'Sac Bunt': 1, 'Pop Out': 1, 'Caught Stealing 2B': 0, 'Fielders Choice Out': 1, 'Stolen Base 2B': 0,
                'Sac Fly': 1, 'Groundout': 1, 'Home Run': 1, 'Caught Stealing 3B': 0, 'Intent Walk': 1, 'Double Play': 1,
                'Grounded Into DP': 1, 'Forceout': 1, 'Lineout': 1, 'Single': 1, 'Triple': 1, 'Wild Pitch': 0, 'Double': 1,
                'Strikeout': 1, 'Bunt Groundout': 1, 'Bunt Pop Out': 1}

pa_events = [event for event, flag in pa_flag_dict.items() if flag == 1]

hit_flag_dict = {'Walk':0, 'Field Error': 0, 'Caught Stealing Home': 0, 'Strikeout Double Play': 0,
                'Sac Fly Double Play': 0,   'Hit By Pitch': 0, 'Runner Out': 0, 'Triple Play': 0, 
                'Field Out': 0, 'Flyout': 0, 'Bunt Lineout': 0, 'Catcher Interference': 0, 'Fielders Choice': 0,
                'Sac Bunt': 0, 'Pop Out': 0, 'Caught Stealing 2B': 0, 'Fielders Choice Out': 0, 'Stolen Base 2B': 0,
                'Sac Fly': 0, 'Groundout': 0, 'Home Run': 1, 'Caught Stealing 3B': 0, 'Intent Walk': 0, 'Double Play': 0,
                'Grounded Into DP': 0, 'Forceout': 0, 'Lineout': 0, 'Single': 1, 'Triple': 1, 'Wild Pitch': 0, 'Double': 1,
                'Strikeout': 0, 'Bunt Groundout': 0, 'Bunt Pop Out': 0}

hit_events = [event for event, flag in hit_flag_dict.items() if flag == 1]

obp_denom_dict = {
    event: 1 for event in ab_flag_dict if ab_flag_dict[event] == 1  # ABs
}
# Include BB, HBP, SF explicitly
for event in ["Walk", "Intent Walk", "Hit By Pitch", "Sac Fly"]:
    obp_denom_dict[event] = 1

# Exclude Sac Bunt if it was included
if "Sac Bunt" in obp_denom_dict:
    obp_denom_dict["Sac Bunt"] = 0

# Create a list of events to use in counting
obp_denom_events = [event for event, flag in obp_denom_dict.items() if flag == 1]

In [138]:
data = pl.read_csv("2025_data.csv")

batters = pl.read_csv("batter_id.csv")

In [ ]:
player_name = "Wyatt Langford"

player_data = data.filter(pl.col("batter_name") == player_name)

player_ab_data = player_data.filter(pl.col("event").is_in(ab_events))

player_pa_data = player_data.filter(pl.col("event").is_in(pa_events))

player_hit_data = player_data.filter(pl.col("event").is_in(hit_events))

'''bughit = player_data.filter((pl.col("pitch_description") == "Knuckle Curve") & (pl.col("event").is_in(hit_events)))
bugab = player_data.filter((pl.col("pitch_description") == "Knuckle Curve") & (pl.col("event").is_in(ab_events)))

hits = bughit.height
abs = bugab.height

print(hits, abs)

bugab'''

2 7


game_id,game_date,batter_id,batter_name,batter_hand,batter_team,batter_team_id,pitcher_id,pitcher_name,pitcher_hand,pitcher_team,pitcher_team_id,ab_number,inning,play_description,play_code,in_play,is_strike,is_swing,is_whiff,is_out,is_ball,is_review,pitch_type,pitch_description,strikes,balls,outs,strikes_after,balls_after,outs_after,start_speed,end_speed,sz_top,sz_bot,x,y,…,px,pz,vx0,vy0,vz0,x0,y0,z0,zone,type_confidence,plate_time,extension,spin_rate,spin_direction,vb,ivb,hb,launch_speed,launch_angle,launch_distance,launch_location,trajectory,hardness,hit_x,hit_y,index_play,play_id,start_time,end_time,is_pitch,type_type,type_ab,event,event_type,rbi,away_score,home_score
i64,str,i64,str,str,str,i64,i64,str,str,str,i64,i64,i64,str,str,bool,bool,bool,bool,bool,bool,bool,str,str,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,i64,i64,f64,f64,f64,f64,i64,i64,i64,str,str,f64,f64,i64,str,str,str,bool,str,str,str,str,i64,i64,i64
777934,"""5/13/2025""",694671,"""Wyatt Langford""","""R""","""TEX""",140,607536,"""Kyle Freeland""","""L""","""COL""",115,21,3,"""In play, out(s)""","""X""",true,false,true,null,true,true,false,"""KC""","""Knuckle Curve""",2,2,0,2,2,0,82.3,75.9,3.409754,1.589679,102.5,191.69,…,0.3803,1.7440752,-4.359748,-119.84099,-1.10053,2.490531,50.002903,5.711447,9,2.0,0.457433,6.668361,2483,321,-49.1,-8.6,4.8,104.3,8,150,6,"""line_drive""","""hard""",108.87,144.16,5,"""3e831b75-3af9-375c-9ef8-593ce4…","""2025-05-14T00:52:58.624Z""","""2025-05-14T00:53:05.489Z""",true,"""pitch""","""atBat""","""Lineout""","""field_out""",0,1,3
777646,"""6/4/2025""",694671,"""Wyatt Langford""","""R""","""TEX""",140,669358,"""Shane Baz""","""R""","""TB""",139,1,1,"""In play, no out""","""D""",true,false,true,null,false,false,false,"""KC""","""Knuckle Curve""",2,1,1,2,1,1,85.3,78.9,3.44,1.66,110.77,174.31,…,0.163449,2.387929,2.158318,-124.21026,-0.030689,-1.3149,50.003165,5.974817,5,0.9,0.440538,6.268844,2739,36,-49.8,-12.4,-8.9,96.3,-14,11,6,"""ground_ball""","""medium""",107.22,157.76,3,"""d7eed651-6a86-37ea-86f9-a4b928…","""2025-06-04T23:38:50.671Z""","""2025-06-04T23:38:59.326Z""",true,"""pitch""","""atBat""","""Field Error""","""field_error""",0,0,0
777646,"""6/4/2025""",694671,"""Wyatt Langford""","""R""","""TEX""",140,669358,"""Shane Baz""","""R""","""TB""",139,18,3,"""Called Strike""","""C""",false,true,null,null,true,true,false,"""KC""","""Knuckle Curve""",2,0,2,3,0,2,86.7,80.4,3.440001,1.560931,96.39,150.3,…,0.540683,3.277138,2.986588,-126.339933,2.425588,-1.178084,50.002581,5.935725,3,0.9,0.432469,6.409653,2697,30,-50.0,-14.0,-7.9,null,null,null,null,null,null,null,null,3,"""8973ad2a-65ff-3769-bf98-4c1b1e…","""2025-06-05T00:16:02.363Z""","""2025-06-05T00:16:05.467Z""",true,"""pitch""","""atBat""","""Strikeout""","""strikeout""",0,0,1
777646,"""6/4/2025""",694671,"""Wyatt Langford""","""R""","""TEX""",140,669358,"""Shane Baz""","""R""","""TB""",139,41,5,"""In play, run(s)""","""E""",true,false,true,null,false,false,false,"""KC""","""Knuckle Curve""",2,2,2,2,2,2,85.8,79.4,3.44,1.66,110.21,179.76,…,0.178052,2.18594,2.260868,-124.996711,-0.394649,-1.445219,50.001625,5.808248,8,0.89,0.437973,6.498596,2711,26,-48.4,-11.3,-10.3,103.1,14,227,7,"""line_drive""","""hard""",44.78,114.99,5,"""694fdb6b-2c33-33e8-925a-9c19bc…","""2025-06-05T01:07:50.273Z""","""2025-06-05T01:08:04.234Z""",true,"""pitch""","""atBat""","""Double""","""double""",1,3,5
777180,"""7/8/2025""",694671,"""Wyatt Langford""","""R""","""TEX""",140,667755,"""José Soriano""","""R""","""LAA""",108,26,4,"""In play, run(s)""","""E""",true,false,true,null,false,false,false,"""KC""","""Knuckle Curve""",0,0,1,0,0,1,85.4,78.8,3.48,1.66,103.52,176.35,…,0.3535519,2.312229,0.66552,-124.365805,-1.527183,-0.967051,50.003298,6.043649,6,0.9,0.440548,6.60363,2565,25,-43.7,-6.2,-14.8,88.9,0,47,7,"""ground_ball""","""medium""",42.51,112.54,1,"""1565110d-d56f-3213-bb45-d02fd4…","""2025-07-09T02:25:34.547Z""","""2025-07-09T02:25:45.219Z""",true,

In [140]:
# Filter data for the player
player_data = data.filter(pl.col("batter_name") == player_name)

# Helper function for counting events
def count_player_events(events):
    return player_data.filter(pl.col("event").is_in(events)).height

# Count ABs, Hits, OBP denominator (exclude Sac Bunt)
num_ab = count_player_events(ab_events)
num_hits = count_player_events(hit_events)
obp_denom = count_player_events(obp_denom_events)

# Batting Average
batting_average = round(num_hits / num_ab if num_ab > 0 else 0, 3)

# On-Base Percentage
on_base_percentage = round(
    (num_hits + count_player_events(["Walk","Intent Walk" ,"Hit By Pitch"])) / obp_denom if obp_denom > 0 else 0,
    3
)

# Slugging Percentage
slugging_percentage = round(
    (1*count_player_events(["Single"]) + 2*count_player_events(["Double"]) +
     3*count_player_events(["Triple"]) + 4*count_player_events(["Home Run"])) / num_ab if num_ab > 0 else 0,
    3
)

# OPS (On-base Plus Slugging)
ops = round(on_base_percentage + slugging_percentage, 3)

# Output
print(f"{player_name} - AVG: {batting_average}, OBP: {on_base_percentage}, SLG: {slugging_percentage}, OPS: {ops}")


Wyatt Langford - AVG: 0.241, OBP: 0.344, SLG: 0.431, OPS: 0.775


In [ ]:
fastball_dict = {'Four-Seam Fastball': 1, 'Sinker': 1, 'Cutter': 1}

offspeed_dict = {'Splitter': 1, 'Changeup': 1, 'Forkball': 1, 'Screwball': 1}

breaking_dict = {'Curveball': 1, 'Knuckle Ball': 1, 'Knuckle Curve': 1, 'Slider': 1, 
                 'Sweeper': 1, 'Slurve': 1, 'Slow Curve': 1}

In [160]:
# Helper function to calculate pitch group stats
def calculate_pitch_group_stats(pitch_dict):
    def count_events(events):
        return player_data.filter(
            pl.col("pitch_description").is_in(pitch_dict) &
            pl.col("event").is_in(events)
        ).height
    
    # ABs, Hits, and OBP denominator (exclude sac bunts)
    num_ab = count_events(ab_events)
    num_hits = count_events(hit_events)
    obp_denom = count_events(obp_denom_events)
    
    # Batting Average
    avg = round(num_hits / num_ab if num_ab > 0 else 0, 3)
    
    # On-Base Percentage
    obp = round(
        (num_hits + count_events(["Walk", "Intent Walk", "Hit By Pitch"])) / obp_denom if obp_denom > 0 else 0,
        3
    )
    
    # Slugging Percentage
    slg = round(
        (1*count_events(["Single"]) + 2*count_events(["Double"]) +
         3*count_events(["Triple"]) + 4*count_events(["Home Run"])) / num_ab if num_ab > 0 else 0,
        3
    )

    ops = round(obp + slg, 3)

    return {"AVG": avg, "OBP": obp, "SLG": slg, "OPS": ops}

# Calculate stats for each pitch group
fastball_stats = calculate_pitch_group_stats(fastball_dict)
breaking_stats = calculate_pitch_group_stats(breaking_dict)
offspeed_stats = calculate_pitch_group_stats(offspeed_dict)

print(f"{player_name} - AVG: {batting_average}, OBP: {on_base_percentage}, SLG: {slugging_percentage}, OPS: {ops}")
print("Fastball Stats:", fastball_stats)
print("Breaking Ball Stats:", breaking_stats)
print("Offspeed Stats:", offspeed_stats)


Wyatt Langford - AVG: 0.241, OBP: 0.344, SLG: 0.431, OPS: 0.775
Fastball Stats: {'AVG': 0.248, 'OBP': 0.359, 'SLG': 0.483, 'OPS': 0.842}
Breaking Ball Stats: {'AVG': 0.252, 'OBP': 0.321, 'SLG': 0.366, 'OPS': 0.687}
Offspeed Stats: {'AVG': 0.178, 'OBP': 0.286, 'SLG': 0.288, 'OPS': 0.574}
